# 文本分类实例

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

主要还是降低显存的方式：
1. 梯度累计
2. 梯度检查点
3. Adafactor，显存节省率根据实现细节大约在30%~50%
4. 模型冻结
5. 数据规模，如batch size和max length

## Step1 导入相关包

In [2]:
# 注意使用不同任务模型：AutoModelForSequenceClassification只对整个句子做一个分类
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments 
from datasets import load_dataset

/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 加载数据集

In [3]:
dataset = load_dataset("csv", data_files="./ChnSentiCorp_htl_all.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

## Step3 划分数据集

In [4]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

## Step4 数据集预处理

In [5]:
import torch

tokenizer = AutoTokenizer.from_pretrained("/data/PLM/chinese-macbert-base")

def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=32, truncation=True, padding="max_length")
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:  43%|████▎     | 3000/6988 [00:00<00:00, 9101.08 examples/s]

Map: 100%|██████████| 777/777 [00:00<00:00, 10791.77 examples/s]


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 777
    })
})

In [6]:
print(tokenizer.decode(tokenized_datasets["train"][0]["input_ids"]))
print(tokenized_datasets["train"][0]["labels"]) # labels当然也可以是常数标签
print(tokenizer.decode(tokenized_datasets["train"][1]["input_ids"]))
print(tokenized_datasets["train"][1]["labels"])

[CLS] 房 间 设 施 还 是 很 先 进 的 ， 自 动 化 程 度 相 当 高 。 但 是 我 最 喜 欢 的 还 是 演 [SEP]
1
[CLS] 因 为 之 前 住 过 这 里 ， 房 间 非 常 非 常 大 ， 所 以 这 次 又 选 择 了 这 边 ， 但 是 [SEP]
1


## Step5 创建模型

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("/data/PLM/chinese-macbert-base")
model # 输入最终分类器前会经过BertPooler层，应该就是在那里处理了[CLS]与其他tkoens的输出！

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /data/PLM/chinese-macbert-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

## Step6 创建评估函数

In [8]:
import evaluate # 可以直接调用评估函数，不用自己写了

acc_metric = evaluate.load("/data/daiyw/Compare/evaluate/metrics/accuracy")
f1_metric = evaluate.load("/data/daiyw/Compare/evaluate/metrics/f1")
# 涵盖的指标包括但不限于accuracy/precision/recall/f1/bleu/rouge/roc_auc/perplexity/mse/mean_iou，可以去/data/daiyw/Compare/evaluate/metrics中查看

In [9]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels) # {'accuracy': 0.x}
    f1 = f1_metric.compute(predictions=predictions, references=labels) # {'f1': 0.x}
    acc.update(f1) # acc和f1以字典形式输出，所以这里更新字典
    return acc

## Step7 创建TrainingArguments

In [10]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=1,   # 训练时的batch_size
                               gradient_accumulation_steps=32,  # *** 梯度累加 ***
                               gradient_checkpointing=True,     # *** 梯度检查点 ***
                               optim="adafactor",               # *** adafactor优化器 *** 
                               per_device_eval_batch_size=1,    # 验证时的batch_size
                               num_train_epochs=1,              # 训练轮数
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数，这个之前都没有注意！
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型
train_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=32,
gradient_checkpointing=True,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_always_push=False,
hub_mod

## Step8 创建Trainer

In [11]:
from transformers import DataCollatorWithPadding # 与DataCollatorForSeq2Seq的不同之处在于这里只考虑对input_ids和attention_mask做padding

# *** 参数冻结 *** 
for name, param in model.bert.named_parameters():
    param.requires_grad = False

trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer), # 此时对labels不会做填充！
                  compute_metrics=eval_metric)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Step9 模型训练

In [12]:
trainer.train() # eval_metric中附加的评价指标也会出现在这里

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


Epoch,Training Loss,Validation Loss,Accuracy,F1
0,0.634500,0.630556,0.679537,0.809195


TrainOutput(global_step=218, training_loss=0.6317042757611756, metrics={'train_runtime': 147.1527, 'train_samples_per_second': 47.488, 'train_steps_per_second': 1.481, 'total_flos': 114716420136960.0, 'train_loss': 0.6317042757611756, 'epoch': 1.0})

In [13]:
trainer.evaluate(tokenized_datasets["test"]) # eval_metric中附加的评价指标也会出现在这里，注意键会相应修改

{'eval_loss': 0.6305564641952515,
 'eval_accuracy': 0.6795366795366795,
 'eval_f1': 0.8091954022988506,
 'eval_runtime': 10.9568,
 'eval_samples_per_second': 70.915,
 'eval_steps_per_second': 70.915,
 'epoch': 1.0}

In [14]:
trainer.predict(tokenized_datasets["test"]) # eval_metric中附加的评价指标也会出现在这里，注意键会相应修改

PredictionOutput(predictions=array([[-0.10445721,  0.7390017 ],
       [-0.08435625,  0.5114457 ],
       [-0.2817772 ,  0.8036922 ],
       ...,
       [ 0.02105268,  0.69490665],
       [ 0.03863226,  0.7115276 ],
       [-0.22607678,  0.5229539 ]], dtype=float32), label_ids=array([1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1

## Step10 模型预测

In [15]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()} # encode后别忘了处理成字典形式，键为"input_ids"、"attention_mask"、"labels"等（这里是测试所以不需要加"labels"）
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [16]:
from transformers import pipeline

model.config.id2label = id2_label # 可以用这个参数将输出直接转化为需要的标签
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [17]:
pipe(sen)

[{'label': '好评！', 'score': 0.7148010730743408}]